In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import LabelEncoder


# Zadanie 1:  

## Email Spam

 Masz dane o 12 emailach z informacją czy to spam czy nie:

 **Zadania do wykonania:**

**a) Ręczne obliczenia**
1. Oblicz prawdopodobieństwa a priori: P(Spam=TAK) i P(Spam=NIE)
2. Dla każdej cechy oblicz prawdopodobieństwa warunkowe
3. Przewidź klasę dla nowego emaila:
```
   Słowo_1 = 'darmowy'
   Słowo_2 = 'wygrana'  
   Wykrzyknik = 'TAK'
```

Oblicz prawdopodobieństwa dla obu klas (TAK lub NIE) i znormalizuj

**b) Implementacja w Python**

1. Zaimplementuj obliczenia z punktu a) w Python (bez sklearn)
2. Porównaj wyniki z ręcznymi obliczeniami

**c) Sklearn**

1. Użyj `CategoricalNB` z sklearn do wytrenowania modelu
2. Porównaj wyniki z własnymi obliczeniami
3. Wyjaśnij różnice (jeśli są)

In [42]:
data_spam = {
    'Słowo_1': ['darmowy', 'darmowy', 'spotkanie', 'raport', 'oferta', 'darmowy',
                'spotkanie', 'oferta', 'raport', 'darmowy', 'spotkanie', 'oferta'],
    'Słowo_2': ['wygrana', 'wygrana', 'jutro', 'kwartalny', 'specjalna', 'rabat',
                'dziś', 'limitowana', 'miesięczny', 'rabat', 'pilne', 'wyjątkowa'],
    'Wykrzyknik': ['TAK', 'TAK', 'NIE', 'NIE', 'TAK', 'TAK',
                   'NIE', 'TAK', 'NIE', 'TAK', 'NIE', 'TAK'],
    'Spam': ['TAK', 'TAK', 'NIE', 'NIE', 'NIE', 'TAK',
             'NIE', 'NIE', 'NIE', 'TAK', 'NIE', 'NIE']
}
df = pd.DataFrame(data_spam)
print(df.to_string(index=True))


print("\nA) Prawdopodobieństwo a priori:")

class_counts = df['Spam'].value_counts()
total = len(df)
priors = class_counts / total

print(f"Liczba przypadków TAK: {class_counts['TAK']}")
print(f"Liczba przypadków NIE: {class_counts['NIE']}")
print(f"Razem: {total}")
print(f"P(Spam = TAK) = {class_counts['TAK']}/{total} = {priors['TAK']:.3f}")
print(f"P(Spam = NIE) = {class_counts['NIE']}/{total} = {priors['NIE']:.3f}")



print("\nPrawdopodobieństwo warunkowe: ")
features = ['Słowo_1', 'Słowo_2', 'Wykrzyknik']
conditionals = {}

for feature in features:
    print(f"\nCecha: {feature.upper()}")
    conditionals[feature] = {}

    for class_val in ['TAK', 'NIE']:
        class_data = df[df['Spam'] == class_val]
        class_count = len(class_data)

        print(f"\nDla klasy {class_val} ({class_count} przypadków):")
        print("-" * 40)

        conditionals[feature][class_val] = {}
        feature_counts = class_data[feature].value_counts()

        for feature_val in df[feature].unique():
            count = feature_counts.get(feature_val, 0)
            prob = count / class_count
            conditionals[feature][class_val][feature_val] = prob

            print(f"  {feature_val:12s}: {count} przypadków → "
                  f"P({feature}={feature_val} | {class_val}) = "
                  f"{count}/{class_count} = {prob:.3f}")

print("Klasa nowego maila: ")

new_record = {
    'Słowo_1': 'darmowy',
    'Słowo_2': 'wygrana',
    'Wykrzyknik': 'TAK'
}

print(f"\nNowy rekord:")
for key, val in new_record.items():
    print(f"  {key}: {val}")
print()

scores = {}

for class_val in ['TAK', 'NIE']:
    print(f"Obliczenia dla klasy: {class_val}")


    score = priors[class_val]
    print(f"\n1. Prior: P({class_val}) = {score:.3f}")

    step = 2
    for feature, value in new_record.items():
        prob = conditionals[feature][class_val].get(value, 0)
        print(f"{step}. P({feature}={value} | {class_val}) = {prob:.3f}")
        score *= prob
        print(f"   Score po mnożeniu: {score:.6f}")
        step += 1

    print(f"\nWynik końcowy dla {class_val}: {score:.6f}")
    scores[class_val] = score

total_score = sum(scores.values())
print(f"\nSuma scores: {scores['TAK']:.6f} + {scores['NIE']:.6f} = {total_score:.6f}")
print()

normalized = {k: v/total_score for k, v in scores.items()}

print("Prawdopodobieństwa po normalizacji:")
for class_val in ['TAK', 'NIE']:
    percentage = normalized[class_val] * 100
    print(f"  P({class_val} | cechy) = {scores[class_val]:.6f} / {total_score:.6f} = "
          f"{normalized[class_val]:.3f} = {percentage:.1f}%")


prediction = max(normalized, key=normalized.get)
print(f"Predykcja: {prediction} z pewnością {normalized[prediction]*100:.1f}%")



      Słowo_1     Słowo_2 Wykrzyknik Spam
0     darmowy     wygrana        TAK  TAK
1     darmowy     wygrana        TAK  TAK
2   spotkanie       jutro        NIE  NIE
3      raport   kwartalny        NIE  NIE
4      oferta   specjalna        TAK  NIE
5     darmowy       rabat        TAK  TAK
6   spotkanie        dziś        NIE  NIE
7      oferta  limitowana        TAK  NIE
8      raport  miesięczny        NIE  NIE
9     darmowy       rabat        TAK  TAK
10  spotkanie       pilne        NIE  NIE
11     oferta   wyjątkowa        TAK  NIE

A) Prawdopodobieństwo a priori:
Liczba przypadków TAK: 4
Liczba przypadków NIE: 8
Razem: 12
P(Spam = TAK) = 4/12 = 0.333
P(Spam = NIE) = 8/12 = 0.667

Prawdopodobieństwo warunkowe: 

Cecha: SŁOWO_1

Dla klasy TAK (4 przypadków):
----------------------------------------
  darmowy     : 4 przypadków → P(Słowo_1=darmowy | TAK) = 4/4 = 1.000
  spotkanie   : 0 przypadków → P(Słowo_1=spotkanie | TAK) = 0/4 = 0.000
  raport      : 0 przypadków → P(Słowo_1=

In [43]:
print("Implementacja własna (bez sklearn)")

scores_b = {}

for class_val in ['TAK', 'NIE']:
    score = priors[class_val]

    for feature, value in new_record.items():
        score *= conditionals[feature][class_val].get(value, 0)

    scores_b[class_val] = score

total_b = sum(scores_b.values())

normalized_b = {c: scores_b[c] / total_b if total_b > 0 else 0
                 for c in scores_b}

print("Implementacja bez sklearn:")
print(f"Scores:")
print(f"  TAK: {scores_b['TAK']}")
print(f"  NIE: {scores_b['NIE']}")

print("\nPo normalizacji:")
print(f"  TAK: {normalized_b['TAK']:.3f}  ({normalized_b['TAK']*100:.1f}%)")
print(f"  NIE: {normalized_b['NIE']:.3f}  ({normalized_b['NIE']*100:.1f}%)")
print("Wynik jest taki sam jak w przypadku ręcznych obliczeń")

Implementacja własna (bez sklearn)
Implementacja bez sklearn:
Scores:
  TAK: 0.16666666666666666
  NIE: 0.0

Po normalizacji:
  TAK: 1.000  (100.0%)
  NIE: 0.000  (0.0%)
Wynik jest taki sam jak w przypadku ręcznych obliczeń


In [47]:
le_dict = {}
features = ['Słowo_1', 'Słowo_2', 'Wykrzyknik']
X_encoded = df[features].copy()

for col in features:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(df[col])
    le_dict[col] = le

le_target = LabelEncoder()
y_encoded = le_target.fit_transform(df['Spam'])

model = CategoricalNB()
model.fit(X_encoded, y_encoded)

new_record = {
    'Słowo_1': 'darmowy',
    'Słowo_2': 'wygrana',
    'Wykrzyknik': 'TAK'
}

new_record_encoded = []
for feature in features:
    value = new_record[feature]
    encoded_value = le_dict[feature].transform([value])[0]
    new_record_encoded.append(encoded_value)

new_record_array = np.array([new_record_encoded])

prediction = model.predict(new_record_array)
probabilities = model.predict_proba(new_record_array)

print("WImplementacja z sklearn")

print(f"Predykcja: {le_target.inverse_transform(prediction)[0]}")
print(f"Prawdopodobieństwa: TAK={probabilities[0][1]:.3f}, NIE={probabilities[0][0]:.3f}")
print("\nRóżnica wynika z użycia smoothingu (α = 1) w sklearn. Sklearn nie pozwala aby prawdopodobieństwo warunkowe wynosiło 0")

WImplementacja z sklearn
Predykcja: TAK
Prawdopodobieństwa: TAK=0.968, NIE=0.032

Różnica wynika z użycia smoothingu (α = 1) w sklearn.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
